# Teams: League Table & Scores Scraper

### Current League Table

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def League_Table_Scraper(url):
    # getting soup from the url
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    league_table = soup.find("div", {'class': 'table_outer_container'}).div.table
    # creating empty lists for looping
    rank = []
    club = []
    games = []
    wins = []
    draws = []
    losses = []
    goals_for = []
    goals_against = []
    goal_diff = []
    points = []
    last_5 = []
    attendance_per_g = []
    top_team_scorers = []
    top_keeper = []
    notes = []
    # looping over different teams (rows)
    for team in league_table.find_all("tbody"):
        rows = team.find_all("tr")
        # looping over different variables (columns) and writing into lists
        for row in rows:
            rank.append(row.find("th", {'data-stat': 'rank'}).text)
            club.append(row.find("td", {'data-stat': 'squad'}).text)
            games.append(row.find("td", {'data-stat': 'games'}).text)
            wins.append(row.find("td", {'data-stat': 'wins'}).text)
            draws.append(row.find("td", {'data-stat': 'draws'}).text)
            losses.append(row.find("td", {'data-stat': 'losses'}).text)
            goals_for.append(row.find("td", {'data-stat': 'goals_for'}).text)
            goals_against.append(row.find("td", {'data-stat': 'goals_against'}).text)
            goal_diff.append(row.find("td", {'data-stat': 'goal_diff'}).text)
            points.append(row.find("td", {'data-stat': 'points'}).text)
            last_5.append(row.find("td", {'data-stat': 'last_5'}).text)
            attendance_per_g.append(row.find("td", {'data-stat': 'attendance_per_g'}).text)
            top_team_scorers.append(row.find("td", {'data-stat': 'top_team_scorers'}).text)
            top_keeper.append(row.find("td", {'data-stat': 'top_keeper'}).text)
            notes.append(row.find("td", {'data-stat': 'notes'}).text)
    # creating a dataframe by concatenating the lists
    table = pd.DataFrame({"Rank": rank, "Team": club, "Games": games, "Wins": wins, "Draws": draws, "Losses": losses, "Goals for": goals_for, "Goals against": goals_against, "Goal Difference": goal_diff, "Points": points, "Last 5 Games": last_5, "Attendance per Game": attendance_per_g, "Top Team Scorers": top_team_scorers, "Top Goalkeeper": top_keeper, "Notes": notes})
    return table

In [3]:
url = "https://fbref.com/en/comps/66/Czech-First-League-Stats"
table = League_Table_Scraper(url)
table.to_csv(r"C:\Users\Honza Stuchlík\Documents\IES\Data Processing in Python\Czech-Football-League\league_table.csv", index = False)
table

,Rank,Team,Games,Wins,Draws,Losses,Goals for,Goals against,Goal Difference,Points,Last 5 Games,Attendance per Game,Top Team Scorers,Top Goalkeeper,Notes
0,1,Slavia Prague,13,11,2,0,36,4,+32,35,D W W W W,"2,308",Abdallah Sima - 7,Ondřej Kolář,
1,2,Jablonec,14,9,2,3,26,15,+11,29,W W D W W,825,Ivan Schranz - 6,Jan Hanuš,
2,3,Sparta Prague,13,9,1,3,29,18,+11,28,W L W W D,"1,652",Lukáš Juliš - 10,Milan Heča,
3,4,Sigma Olomouc,14,5,7,2,20,15,+5,22,D D D D L,936,"Martin Nešpor, David Houska - 3",Aleš Mandous,
4,5,FK Pardubice,14,6,4,4,15,15,0,22,D W L D W,673,David Huf - 5,Marek Boháč,
5,6,Slovácko,13,6,3,4,23,15,+8,21,D L L W W,569,Vlastimil Daníček - 4,Vít Nemrava,
6,7,Slovan Liberec,13,6,3,4,20,15,+5,21,D L W W D,"1,124",Michael Rabušic - 6,Filip Nguyen,
7,8,Baník Ostrava,12,6,3,3,18,10,+8,21,W W W D W,"2,210",Dyjan Carlos De Azevedo - 6,Jan Laštůvka,
8,9,Karviná,14,5,5,4,14,18,-4,20,W D D W L,844,"Kristi Qose, Michal Papadopulos - 3",Petr Bolek,
9,10,Viktoria Plzeň,14,6,2,6,28,21,+7,20,L L W L L,"1,523","Aleš Čermák, Jean-David Beauguel - 6",Aleš Hruška,


### Scores & Fixtures

We wrote two different functions for scraping scores from seasons with regular rounds only and from seasons with extra rounds called championship or relegation because the html code on the web page has various structures for these types of seasons, so one function would not work for all we need.

In [4]:
# for regular seasons only (without championship/relegation rounds)
def Scores_Scraper_regular(url):
    
    # getting soup from the url
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    scores_table = soup.find("div", {'class': 'table_outer_container'}).div.table

    gameweek = []
    dayofweek = []
    date = []
    time = []
    squad_a = []
    score = []
    squad_b = []
    attendance = []
    venue = []
    referee = []
    match_report = []
    notes = []

    # looping over different teams (rows)
    for game in scores_table.find_all("tbody"):
        rows = game.find_all("tr")
        
        # looping over different variables (columns) and writing into lists
        for row in rows:
            gameweek.append(row.find("th", {'data-stat': 'gameweek'}).text)
            dayofweek.append(row.find("td", {'data-stat': 'dayofweek'}).text)
            date.append(row.find("td", {'data-stat': 'date'}).text)
            time.append(row.find("td", {'data-stat': 'time'}).text)
            squad_a.append(row.find("td", {'data-stat': 'squad_a'}).text)
            score.append(row.find("td", {'data-stat': 'score'}).text)
            squad_b.append(row.find("td", {'data-stat': 'squad_b'}).text)
            attendance.append(row.find("td", {'data-stat': 'attendance'}).text)
            venue.append(row.find("td", {'data-stat': 'venue'}).text)
            referee.append(row.find("td", {'data-stat': 'referee'}).text)
            match_report.append(row.find("td", {'data-stat': 'match_report'}).text)
            notes.append(row.find("td", {'data-stat': 'notes'}).text)
            
    # creating a dataframe by concatenating the lists
    scores = pd.DataFrame({"Game Week": gameweek, "Weekday": dayofweek, "Date": date, "Time": time, "Home Team": squad_a, "Score": score, "Away Team": squad_b, "Attendance": attendance, "Venue": venue, "Referee": referee, "Match Report": match_report, "Notes": notes})
    
    return scores

In [5]:
url = "https://fbref.com/en/comps/66/schedule/Czech-First-League-Scores-and-Fixtures"
Scores_Scraper_regular(url)

,Game Week,Weekday,Date,Time,Home Team,Score,Away Team,Attendance,Venue,Referee,Match Report,Notes
0,1,Fri,2020-08-21,18:00,Viktoria Plzeň,3–1,Opava,"2,813",Doosan Arena,Alex Denev,Match Report,
1,1,Sat,2020-08-22,17:00,Fastav Zlín,1–2,Slovácko,"1,282",Stadion Letná,Pavel Královec,Match Report,
2,1,Sat,2020-08-22,17:00,Příbram,1–3,Teplice,"1,350",Energon Aréna,Ondřej Berka,Match Report,
3,1,Sat,2020-08-22,17:00,Sigma Olomouc,1–0,Slovan Liberec,"2,216",Andrův stadion,Paval Julínek,Match Report,
4,1,Sat,2020-08-22,19:30,Zbrojovka Brno,1–4,Sparta Prague,"2,500",Městský fotbalový stadion Srbská,Pavel Franek,Match Report,
...,...,...,...,...,...,...,...,...,...,...,...,...
338,34,Fri,2021-05-28,,Slovácko,,Fastav Zlín,,Městský fotbalový stadion Miroslava Vale...,,Head-to-Head,
339,34,Fri,2021-05-28,,Slavia Prague,,České Budĕjov.,,Sinobo Stadium,,Head-to-Head,
340,34,Fri,2021-05-28,,FK Pardubice,,Jablonec,,,,Head-to-Head,
341,,,,,,,,,,,,


In [6]:
# for seasons with regular rounds and championship/relegation rounds
def Scores_Scraper_irregular(url):
    
    # getting soup from the url
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    scores_table = soup.find("div", {'class': 'table_outer_container'}).div.table

    gameweek = []
    dayofweek = []
    date = []
    time = []
    squad_a = []
    score = []
    squad_b = []
    attendance = []
    venue = []
    referee = []
    match_report = []
    notes = []

    # looping over different teams (rows)
    for game in scores_table.find_all("tbody"):
        rows = game.find_all("tr")
        
        # looping over different variables (columns) and writing into lists
        for row in rows:
            gameweek.append(row.find("td", {'data-stat': 'gameweek'}).text)
            dayofweek.append(row.find("td", {'data-stat': 'dayofweek'}).text)
            date.append(row.find("td", {'data-stat': 'date'}).text)
            time.append(row.find("td", {'data-stat': 'time'}).text)
            squad_a.append(row.find("td", {'data-stat': 'squad_a'}).text)
            score.append(row.find("td", {'data-stat': 'score'}).text)
            squad_b.append(row.find("td", {'data-stat': 'squad_b'}).text)
            attendance.append(row.find("td", {'data-stat': 'attendance'}).text)
            venue.append(row.find("td", {'data-stat': 'venue'}).text)
            referee.append(row.find("td", {'data-stat': 'referee'}).text)
            match_report.append(row.find("td", {'data-stat': 'match_report'}).text)
            notes.append(row.find("td", {'data-stat': 'notes'}).text)
            
    # creating a dataframe by concatenating the lists
    scores = pd.DataFrame({"Game Week": gameweek, "Weekday": dayofweek, "Date": date, "Time": time, "Home Team": squad_a, "Score": score, "Away Team": squad_b, "Attendance": attendance, "Venue": venue, "Referee": referee, "Match Report": match_report, "Notes": notes})
    
    return scores

In [7]:
url = "https://fbref.com/en/comps/66/3226/schedule/2019-2020-Czech-First-League-Scores-and-Fixtures"
Scores_Scraper_irregular(url)

,Game Week,Weekday,Date,Time,Home Team,Score,Away Team,Attendance,Venue,Referee,Match Report,Notes
0,1,Fri,2019-07-12,18:00,Jablonec,2–0,Bohemians 1905,"2,612",Stadion Střelnice,Pavel Franek,Match Report,
1,1,Sat,2019-07-13,17:00,Příbram,1–1,Teplice,"2,862",Energon Aréna,Paval Julínek,Match Report,
2,1,Sat,2019-07-13,17:00,Baník Ostrava,1–2,Slovan Liberec,"7,542",Městský stadion - Vítkovice Aréna,Ondřej Berka,Match Report,
3,1,Sat,2019-07-13,19:30,Viktoria Plzeň,3–1,Sigma Olomouc,"9,611",Doosan Arena,Ondřej Pechanec,Match Report,
4,1,Sun,2019-07-14,16:30,České Budĕjov.,0–1,Opava,"4,381",Fotbalový stadion Střelecký ostrov,Ondřej Ginzel,Match Report,
...,...,...,...,...,...,...,...,...,...,...,...,...
311,4,Thu,2020-07-23,18:00,Fastav Zlín,,Karviná,,Stadion Letná,,,Match Postponed
312,,,,,,,,,,,,
313,5,Sun,2020-07-26,17:00,Sigma Olomouc,,Fastav Zlín,,Andrův stadion,,,Match Postponed
314,5,Sun,2020-07-26,17:00,Karviná,,Příbram,,Městský stadion,,,Match Postponed


In [8]:
regular_season_urls = ["https://fbref.com/en/comps/66/1459/schedule/2015-2016-Czech-First-League-Scores-and-Fixtures", "https://fbref.com/en/comps/66/1518/schedule/2016-2017-Czech-First-League-Scores-and-Fixtures", "https://fbref.com/en/comps/66/1623/schedule/2017-2018-Czech-First-League-Scores-and-Fixtures"]
irregular_season_urls = ["https://fbref.com/en/comps/66/2427/schedule/2018-2019-Czech-First-League-Scores-and-Fixtures", "https://fbref.com/en/comps/66/3226/schedule/2019-2020-Czech-First-League-Scores-and-Fixtures", "https://fbref.com/en/comps/66/schedule/Czech-First-League-Scores-and-Fixtures"]
scores_1516 = Scores_Scraper_regular("https://fbref.com/en/comps/66/1459/schedule/2015-2016-Czech-First-League-Scores-and-Fixtures")
scores_1617 = Scores_Scraper_regular("https://fbref.com/en/comps/66/1518/schedule/2016-2017-Czech-First-League-Scores-and-Fixtures")
scores_1718 = Scores_Scraper_regular("https://fbref.com/en/comps/66/1623/schedule/2017-2018-Czech-First-League-Scores-and-Fixtures")
scores_1819 = Scores_Scraper_irregular("https://fbref.com/en/comps/66/2427/schedule/2018-2019-Czech-First-League-Scores-and-Fixtures")
scores_1920 = Scores_Scraper_irregular("https://fbref.com/en/comps/66/3226/schedule/2019-2020-Czech-First-League-Scores-and-Fixtures")
scores_2021 = Scores_Scraper_regular("https://fbref.com/en/comps/66/schedule/Czech-First-League-Scores-and-Fixtures")
scores_dfs = [scores_1516, scores_1617, scores_1718, scores_1819, scores_1920, scores_2021]
scores = pd.DataFrame()
for i in scores_dfs:
    scores = scores.append(i, ignore_index = True, sort = False)

scores.to_csv(r"C:\Users\Honza Stuchlík\Documents\IES\Data Processing in Python\Czech-Football-League\scores.csv", index = False)
scores

,Game Week,Weekday,Date,Time,Home Team,Score,Away Team,Attendance,Venue,Referee,Match Report,Notes
0,1,Fri,2015-07-24,17:30,Viktoria Plzeň,2–1,Slavia Prague,"11,233",Doosan Arena,Pavel Franek,Match Report,
1,1,Fri,2015-07-24,19:00,Vysočina Jihlava,0–0,Sparta Prague,"3,894",Stadion v Jiráskově ulici,Tomas Kocourek,Match Report,
2,1,Sat,2015-07-25,17:00,Příbram,2–3,Jablonec,"4,182",Energon Aréna,Pavel Královec,Match Report,
3,1,Sat,2015-07-25,17:00,Slovácko,4–3,Dukla Prague,"3,726",Městský fotbalový stadion Miroslava Vale...,Zbyněk Proske,Match Report,
4,1,Sat,2015-07-25,17:00,Zbrojovka Brno,2–1,Baník Ostrava,"5,326",Městský fotbalový stadion Srbská,Libor Kovařík,Match Report,
...,...,...,...,...,...,...,...,...,...,...,...,...
1727,34,Fri,2021-05-28,,Slovácko,,Fastav Zlín,,Městský fotbalový stadion Miroslava Vale...,,Head-to-Head,
1728,34,Fri,2021-05-28,,Slavia Prague,,České Budĕjov.,,Sinobo Stadium,,Head-to-Head,
1729,34,Fri,2021-05-28,,FK Pardubice,,Jablonec,,,,Head-to-Head,
1730,,,,,,,,,,,,
